
# 単語の埋め込み (word2vec)

 :ラベル: `sec_word2vec`

自然言語は、意味を表現するために使用される複雑なシステムです。このシステムでは、単語が意味の基本単位となります。名前が示すように、*単語ベクトルは*単語を表すために使用されるベクトルであり、単語の特徴ベクトルまたは表現として考えることもできます。単語を実数ベクトルにマッピングする手法は、*単語埋め込みと*呼ばれます。近年、単語の埋め込みは徐々に自然言語処理の基礎知識になりつつあります。

## ワンホットベクターは悪い選択です

:numref: `sec_rnn-scratch`で単語 (文字は単語) を表すためにワンホット ベクトルを使用しました。辞書内の異なる単語の数 (辞書サイズ) が $N$ で、各単語が $0$ から $N-1$ までの異なる整数 (インデックス) に対応すると仮定します。インデックス $i$ を持つ単語のワンホット ベクトル表現を取得するには、すべて 0 で長さ $N$ ベクトルを作成し、位置 $i$ の要素を 1 に設定します。このようにして、各単語は次のように表現されます。長さ $N$ のベクトルで、ニューラル ネットワークで直接使用できます。

ワンホット ワード ベクトルは構築が簡単ですが、通常は良い選択ではありません。主な理由は、ワンホット ワード ベクトルでは、私たちがよく使う*コサイン類似度*など、異なる単語間の類似度を正確に表現できないことです。ベクトル $\mathbf{x}, \mathbf{y} \in \mathbb{R}^d$ の場合、それらのコサイン類似度はそれらの間の角度のコサインです。

 $$\frac{\mathbf{x}^\top \mathbf{y}}{|\mathbf{x}| |\mathbf{y}|} \in [-1, 1].$$

任意の 2 つの異なる単語のワンホット ベクトル間のコサイン類似度は 0 であるため、ワンホット ベクトルは単語間の類似性をエンコードできません。

## 自己監視型 word2vec

 [word2vec](https://code.google.com/archive/p/word2vec/)ツールは、上記の問題に対処するために提案されました。各単語を固定長のベクトルにマッピングし、これらのベクトルは異なる単語間の類似性と類似関係をより適切に表現できます。 word2vec ツールには、 *skip-gram* :cite: `Mikolov.Sutskever.Chen.ea.2013`と*Continuous Bag of Words* (CBOW) :cite: `Mikolov.Chen.Corrado.ea.2013`という 2 つのモデルが含まれています。意味的に意味のある表現の場合、そのトレーニングは、コーパス内の周囲の単語の一部を使用して単語を予測すると見なすことができる条件付き確率に依存します。教師はラベルのないデータから得られるため、スキップグラムと連続バッグ オブ ワードは両方とも自己教師ありモデルです。

以下では、これら 2 つのモデルとそのトレーニング方法を紹介します。

## スキップグラムモデル

:label: `subsec_skip-gram`

*スキップグラム*モデルは、単語を使用してテキスト シーケンス内で周囲の単語を生成できることを前提としています。例として、テキストシーケンス「the」、「man」、「loves」、「his」、「son」を取り上げます。*中心単語*として「loves」を選択し、コンテキスト ウィンドウのサイズを 2 に設定しましょう。 :numref: `fig_skip_gram`に示すように、中心単語「loves」を指定すると、スキップグラム モデルは*文脈単語*を生成するための条件付き確率を考慮します。 the"、"man"、"his"、および "son"。中心の単語から 2 単語以内の距離にあります。

 $$P(\textrm{"その人"},\textrm{"男"},\textrm{"彼の"},\textrm{"息子"}\mid\textrm{"大好き"}).$$

中心単語が与えられると、文脈単語が独立して生成されると仮定します (つまり、条件付き独立)。この場合、上記の条件付き確率は次のように書き換えることができます。

 $$P(\textrm{"the"}\mid\textrm{"loves"})\cdot P(\textrm{"man"}\mid\textrm{"loves"})\cdot P(\textrm{"彼の"}\mid\textrm{"大好き"})\cdot P(\textrm{"息子"}\mid\textrm{"大好き"}).$$ 

![](../img/skip-gram.svg) :label: `fig_skip_gram`

スキップグラム モデルでは、各単語には、条件付き確率を計算するための 2 つの $d$ 次元ベクトル表現があります。より具体的には、辞書内のインデックス $i$ を持つ単語を $\mathbf{v}_i\in\mathbb{R}^d$ と $\mathbf{u}_i\in\mathbb{R}^ で表します。 d$ は、それぞれ*中心*単語と*文脈*単語として使用される場合の 2 つのベクトルです。中心単語 $w_c$ (辞書内のインデックス $c$ を持つ) が与えられた場合に、任意のコンテキスト ワード $w_o$ (辞書内のインデックス $o$ を持つ) を生成する条件付き確率は、ベクトル ドット積のソフトマックス演算によってモデル化できます。

 $$P(w_o \mid w_c) = \frac{\text{exp}(\mathbf{u} *o^\top \mathbf{v}_c)}{ \sum* {i \in \mathcal{V}} \ text{exp}(\mathbf{u}_i^\top \mathbf{v}_c)},$$ :eqlabel: `eq_skip-gram-softmax`

ここで、語彙インデックスは $\mathcal{V} = {0, 1, \ldots, |\mathcal{V}|-1}$ に設定されます。長さ $T$ のテキスト シーケンスが与えられるとします。タイム ステップ $t$ の単語は $w^{(t)}$ と表されます。任意の中心単語が与えられると、文脈単語が独立して生成されると仮定します。コンテキスト ウィンドウ サイズ $m$ の場合、スキップグラム モデルの尤度関数は、任意の中心単語が与えられた場合にすべてのコンテキスト ワードを生成する確率です。

 $$ \prod_{t=1}^{T} \prod_{-m \leq j \leq m,\ j \neq 0} P(w^{(t+j)} \mid w^{(t) })、$$

ここで、$1$ 未満または $T$ を超えるタイム ステップは省略できます。

### トレーニング

スキップグラム モデルのパラメーターは、語彙内の各単語の中心単語ベクトルと文脈単語ベクトルです。トレーニングでは、尤度関数を最大化する (つまり、最尤推定) ことによってモデル パラメーターを学習します。これは、次の損失関数を最小化することと同じです。

 $$ - \sum_{t=1}^{T} \sum_{-m \leq j \leq m,\ j \neq 0} \text{log}, P(w^{(t+j)} \半ば w^{(t)}).$$

確率的勾配降下法を使用して損失を最小限に抑える場合、反復ごとに短いサブシーケンスをランダムにサンプリングして、このサブシーケンスの (確率的) 勾配を計算し、モデル パラメーターを更新できます。この（確率的）勾配を計算するには、中心単語ベクトルと文脈単語ベクトルに関する対数条件付き確率の勾配を取得する必要があります。一般に、:eqref: `eq_skip-gram-softmax`によれば、中心単語 $w_c$ と文脈単語 $w_o$ の任意のペアに関係する対数条件付き確率は次のようになります。

 $$\log P(w_o \mid w_c) =\mathbf{u} *o^\top \mathbf{v}_c - \log\left(\sum* {i \in \mathcal{V}} \text{exp} (\mathbf{u}_i^\top \mathbf{v}_c)\right).$$ :eqlabel: `eq_skip-gram-log`

微分により、中心単語ベクトル $\mathbf{v}_c$ に対する勾配を次のように取得できます。

 $$\begin{aligned}\frac{\partial \text{log}, P(w_o \mid w_c)}{\partial \mathbf{v} *c}&amp;= \mathbf{u}_o - \frac{\sum* {j \in \mathcal{V}} \exp(\mathbf{u} *j^\top \mathbf{v}_c)\mathbf{u}_j}{\sum* {i \in \mathcal{V}} \ exp(\mathbf{u} *i^\top \mathbf{v}_c)}\&amp;= \mathbf{u}_o - \sum* {j \in \mathcal{V}} \left(\frac{\text{ exp}(\mathbf{u} *j^\top \mathbf{v}_c)}{ \sum* {i \in \mathcal{V}} \text{exp}(\mathbf{u} *i^\top \mathbf {v}_c)}\right) \mathbf{u}_j\&amp;= \mathbf{u}_o - \sum {j \* in \mathcal{V}} P(w_j \mid w_c) \mathbf{u}_j .\end{aligned}$$ :eqlabel: `eq_skip-gram-grad`

 :eqref: `eq_skip-gram-grad`の計算には、$w_c$ を中心単語とする辞書内のすべての単語の条件付き確率が必要であることに注意してください。他の単語ベクトルの勾配も同様に取得できます。

訓練後、辞書内のインデックス $i$ を持つ単語について、単語ベクトル $\mathbf{v}_i$ (中心単語として) と $\mathbf{u}_i$ (文脈単語として) の両方を取得します。自然言語処理アプリケーションでは、通常、スキップグラム モデルの中心単語ベクトルが単語表現として使用されます。

##  Continuous Bag of Words (CBOW) モデル

*Continuous Bag of Words* (CBOW) モデルは、スキップグラム モデルに似ています。スキップグラム モデルとの主な違いは、連続バッグ オブ ワード モデルでは、テキスト シーケンス内の周囲のコンテキスト ワードに基づいて中心単語が生成されると想定していることです。たとえば、同じテキスト シーケンス「the」、「man」、「loves」、「his」、および「son」で、「loves」が中心単語であり、コンテキスト ウィンドウのサイズが 2 である場合、連続した単語のバッグが表示されます。モデルは、コンテキスト単語「the」、「man」、「his」、「son」に基づいて中心単語「loves」を生成する条件付き確率を考慮します ( :numref: `fig_cbow`に示すように)。

 $$P(\textrm{"大好き"}\mid\textrm{"その人"},\textrm{"男"},\textrm{"彼"},\textrm{"息子"}).$$ 

![](http://d2l.ai/_images/cbow.svg) :ラベル: `fig_cbow`

連続バッグオブワードモデルには複数の文脈ワードが存在するため、条件付き確率の計算ではこれらの文脈ワードベクトルが平均化されます。具体的には、辞書内のインデックス $i$ を持つ単語については、$\mathbf{v *} i\in\mathbb{R}^d$ および $\mathbf{u}_i\in\mathbb{R}^d で*表します。 *$ それぞれ *context* 単語と *center* 単語として使用される場合の 2 つのベクトル (意味はスキップグラム モデルで切り替えられます)。周囲のコンテキスト単語 $w {o_1}、\ldots、w_{o_{2m}}$ (インデックス $o_1、\ を含む) を考慮して、任意の中心単語 $w_c$ (辞書内のインデックス $c$ を持つ) を生成する条件付き確率。* ldots、辞書では o_{2m}$) は次のようにモデル化できます。

 $$P(w_c \mid w_{o_1}, \ldots, w_{o_{2m}}) = \frac{\text{exp}\left(\frac{1}{2m}\mathbf{u} *c^ \top (\mathbf{v}* {o_1} + \ldots + \mathbf{v} *{o* {2m}}) \right)}{ \sum_{i \in \mathcal{V}} \text{exp}\ left(\frac{1}{2m}\mathbf{u} *i^\top (\mathbf{v}* {o_1} + \ldots + \mathbf{v} *{o* {2m}}) \right)}.$ $ :eqlabel: `fig_cbow-full`

簡潔にするために、$\mathcal{W} *o= {w* {o_1}, \ldots, w_{o_{2m}}}$ および $\bar{\mathbf{v}} *o = \left(\mathbf{v }* {o_1} + \ldots + \mathbf{v} *{o* {2m}} \right)/(2m)$。次に、:eqref: `fig_cbow-full`次のように簡略化できます。

 *$$P(w_c \mid \mathcal{W} o) = \frac{\exp\left(\mathbf{u}_c^\top \bar{\mathbf{v}}_o\right)}{\sum {* i \in \mathcal{V}} \exp\left(\mathbf{u}_i^\top \bar{\mathbf{v}}_o\right)}.$$

長さ $T$ のテキスト シーケンスが与えられるとします。タイム ステップ $t$ の単語は $w^{(t)}$ と表されます。コンテキスト ウィンドウ サイズ $m$ の場合、連続バッグ オブ ワード モデルの尤度関数は、コンテキスト ワードが与えられた場合にすべての中心ワードを生成する確率です。

 $$ \prod_{t=1}^{T} P(w^{(t)} \mid w^{(tm)}, \ldots, w^{(t-1)}, w^{(t +1)}, \ldots, w^{(t+m)}).$$

### トレーニング

連続バッグオブワードモデルのトレーニングは、スキップグラムモデルのトレーニングとほぼ同じです。連続バッグ オブ ワード モデルの最尤推定は、次の損失関数を最小化することと同じです。

 $$ -\sum_{t=1}^T \text{log}, P(w^{(t)} \mid w^{(tm)}, \ldots, w^{(t-1)}, w^{(t+1)}, \ldots, w^{(t+m)}).$$

注目してください

$$\log,P(w_c \mid \mathcal{W *} o) = \mathbf{u}_c^\top \bar{\mathbf{v}}_o - \log,\left(\sum {i \* in \mathcal{V}} \exp\left(\mathbf{u}_i^\top \bar{\mathbf{v}}_o\right)\right).$$

微分を通じて、任意の文脈単語ベクトル $\mathbf{v}_{o_i}$($i = 1, \ldots, 2m$) に関する勾配を取得できます。

 $$\frac{\partial \log, P(w_c \mid \mathcal{W} *o)}{\partial \mathbf{v}* {o_i}} = \frac{1}{2m} \left(\mathbf{ u} *c - \sum* {j \in \mathcal{V}} \frac{\exp(\mathbf{u *} j^\top \bar{\mathbf{v}}_o)\mathbf{u}_j}{ \sum* {i \in \mathcal{V}} \text{exp}(\ *mathbf{u} i^\top \bar{\mathbf{v}}_o)} \right) = \frac{1}{2m }\left(\mathbf{u}_c - \sum* {j \in \mathcal{V}} P(w_j \mid \mathcal{W}_o) \mathbf{u}_j \right).$$ :eqlabel: `eq_cbow-gradient`

他の単語ベクトルの勾配も同様に取得できます。スキップグラム モデルとは異なり、連続バッグ オブ ワード モデルは通常、単語表現としてコンテキスト ワード ベクトルを使用します。

## まとめ
- 単語ベクトルは単語を表すために使用されるベクトルであり、単語の特徴ベクトルまたは表現として考えることもできます。単語を実数ベクトルにマッピングする手法は、単語埋め込みと呼ばれます。
-  word2vec ツールには、スキップグラム モデルと連続バッグ オブ ワード モデルの両方が含まれています。
- スキップグラム モデルは、単語を使用してテキスト シーケンス内で周囲の単語を生成できることを前提としています。一方、連続的な単語のバッグ モデルは、中心単語が周囲の文脈単語に基づいて生成されることを前提としています。

## 演習
1. 各勾配を計算するための計算の複雑さはどれくらいですか?辞書のサイズが大きい場合、何が問題になるでしょうか?
1. 英語の定型文には、「new york」のように複数の単語で構成されるものもあります。単語ベクトルをトレーニングするにはどうすればよいでしょうか?ヒント: word2vec 論文のセクション 4 を参照してください:cite: `Mikolov.Sutskever.Chen.ea.2013` 。
1. スキップグラム モデルを例として、word2vec の設計について考えてみましょう。スキップグラム モデルの 2 つのワード ベクトルの内積とコサイン類似度との関係は何ですか?同様のセマンティクスを持つ単語のペアについて、(スキップグラム モデルによってトレーニングされた) 単語ベクトルのコサイン類似性が高いのはなぜでしょうか?

[ディスカッション](https://discuss.d2l.ai/t/381)
